# Resources
- Transfomer Adapter example Colab notebooks: https://github.com/Adapter-Hub/adapter-transformers/tree/master/notebooks


# Setup

**TODO:** Create a directory in your google drive called `dl-group-project`. In it,
- Upload the contents of the project repo `dl-group-project`
- Upload the unzipped `data`

## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now we will change directories to the `dl-group-project` folder

**TODO:** Change the below `cd` statement to match the path in your Google Drive to your `dl-group-project` directory

In [2]:
%cd /content/drive/MyDrive/School/semesters/spring_2022/CS_7643_DL/dl-group-project
%ls

/content/drive/MyDrive/School/semesters/spring_2022/CS_7643_DL/dl-group-project
checkpoints/    modelT5Adapter.py    requirements.txt       tf_test.ipynb
data/           modelT5.py           results/               train_adapter.ipynb
dataprep.ipynb  PT_T5_adapter.ipynb  testing_adapter.ipynb  training_output/
dataprovider/   PT_T5.ipynb          testing.ipynb          train.ipynb
data.zip        __pycache__/         testing.txt
examples/       README.md            TF_T5.ipynb


## Install necessary packages

In [3]:
!pip install transformers
!pip install -U adapter-transformers
!pip install datasets
!pip install pytorch_lightning
!pip install sentencepiece

## Check GPU

In [4]:
!nvidia-smi

Thu Apr 28 03:54:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Imports

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# import transformers
# import datasets

from transformers import (
    # AdamW, 
    T5Model, 
    T5ForConditionalGeneration, 
    T5AdapterModel, 
    T5Tokenizer, 
    get_linear_schedule_with_warmup,
    TrainingArguments, 
    AdapterTrainer,
    Trainer
)

from dataprovider.DataProvider import DatasetProvider

# Load Data

In [7]:
# Load data
train_dataset = DatasetProvider('t5-small', 'data/COMBINED', 'train')
test_dataset = DatasetProvider("t5-small", "data/COMBINED", "test")
# Combine into dictionary
dataset = {
    "train": train_dataset,
    "test": test_dataset
}

data/COMBINED/train.tsv


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:195: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


data/COMBINED/test.tsv


# Tokenizer

In [8]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Adapter Model Train

In [9]:
adapter_model = T5AdapterModel.from_pretrained("t5-small")
adapter_model.add_seq2seq_lm_head("lm_head")
adapter_model.add_adapter("paraphrase1")
adapter_model.train_adapter("paraphrase1")

Some weights of the model checkpoint at t5-small were not used when initializing T5AdapterModel: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5AdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5AdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of T5AdapterModel were not initialized from the model checkpoint at t5-small and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Train

## Setup Training

In [14]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-4,
    weight_decay=0.01,
    num_train_epochs=3,
    remove_unused_columns=False
)

trainer = AdapterTrainer(
    model=adapter_model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"]
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


## Run training

In [15]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 58618
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 21984


KeyError: ignored

In [12]:
# Debug training loop
for batch in trainer.get_train_dataloader():
    break

outputs = trainer.model.cpu()(**batch)
loss = outputs.loss
loss.backward()

AttributeError: ignored

In [16]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 5177
  Batch size = 8


AttributeError: ignored

# Save

In [ ]:
adapter_modeel.save_adapter("adapter_paraphrase", "paraphrase")